In [ ]:
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

# Read cvs
gdf = cudf.read_csv('../data/Music.csv')

# Create table
bc.create_table('music', gdf)

# Query
result = bc.sql('SELECT * FROM main.music', ['music']).get()

# Get GDF
result_gdf = result.columns

# Print GDF
print(result_gdf)

In [ ]:
import cudf
from blazingsql import BlazingContext

bc = BlazingContext()

column_names = ['n_nationkey', 'n_name', 'n_regionkey', 'n_comments']
column_types = ['int32', 'int64', 'int32', 'int64']

nation_gdf = cudf.read_csv("../data/nation.psv", delimiter='|',
                           dtype=column_types, names=column_names)

print(nation_gdf)
bc.create_table('nation', nation_gdf)

query = 'select n_nationkey, n_regionkey, n_nationkey + n_regionkey as addition from main.nation'
result_gdf = bc.sql(query).get()

print(query)
print(result_gdf)


In [ ]:
import cudf
from blazingsql import BlazingContext
import pandas as pd

bc = BlazingContext()

column_names = ['n_nationkey', 'n_name', 'n_regionkey', 'n_comments']
column_types = {'n_nationkey': 'int32', 'n_regionkey': 'int64'}
nation_df = pd.read_csv("../data/nation.psv", delimiter='|',
                        dtype=column_types, names=column_names)

print(nation_df)

bc.create_table('nation', nation_df)
query = 'select n_nationkey, n_regionkey, n_nationkey + n_regionkey as addition from main.nation'
result_gdf = bc.sql(query).get()

print(query)
print(result_gdf)


In [ ]:
import cudf
from blazingsql import BlazingContext
import pyarrow as pa

bc = BlazingContext()

arrow_table = pa.RecordBatchStreamReader('../data/gpu.arrow').read_all()
df = arrow_table.to_pandas()
df = df[['swings', 'tractions']]
print(df)

bc.create_table('gpu_info', df)

query = 'select swings+1, tractions+10 from main.gpu_info'
result_gdf = bc.sql(query).get()

print(query)
print(result_gdf)


In [ ]:
from blazingsql import BlazingContext
import cudf
bc = BlazingContext()

# Csv files
column_names = ['diagnosis_result', 'radius', 'texture', 'perimeter']
column_types = ['float32', 'float32', 'float32', 'float32']
bc.create_table('data_00', '/blazingdb/data/cancer_data_00.csv', delimiter=',', dtype=column_types, names=column_names)

# Parquet
column_names = ['perimeter', 'area', 'smoothness', 'compactness']
bc.create_table('data_01_parquet', '/blazingdb/data/cancer_data_01.parquet')
query_01 = bc.sql('SELECT * FROM main.data_01_parquet').get()
gdf_01 = query_01.columns
bc.create_table('data_01', gdf_01)

# Gdf
column_names = ['compactness', 'symmetry', 'fractal_dimension']
column_types = ['float32', 'float32', 'float32', 'float32']
gdf_02= cudf.read_csv('/blazingdb/data/cancer_data_02.csv',delimiter=',', dtype=column_types, names=column_names)
bc.create_table('data_02', gdf_02)

# Query
sql = '''
SELECT a.*, b.area, b.smoothness, c.* from main.data_00 as a
LEFT JOIN main.data_01  as b
ON (a.perimeter = b.perimeter)
LEFT JOIN main.data_02 as c
ON (b.compactness = c.compactness)
'''
join = bc.sql(sql).get()
result = join.columns
print(result)
